In [10]:
import pandas as pd,numpy as np, seaborn as sns, matplotlib.pyplot as plt, joblib
from scipy import stats
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score, confusion_matrix

In [2]:
df = pd.read_csv('../0. Dataset/Training set after splitting.csv')

In [3]:
df.head()

,CustomerID,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,...,IncomeGroup,OwnsMotorcycle,AdjustmentsToCreditRating,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus,Churn
0,3317770,39.79,234.0,55.0,0.25,0.0,1.7,31.0,8.8,6.7,...,0,No,0,Unknown,No,2-High,Suburban,Other,Unknown,No
1,3224894,60.19,551.0,60.0,0.74,0.0,0.3,-88.0,-10.2,2.7,...,7,No,0,150,No,2-High,Suburban,Crafts,Yes,No
2,3231266,85.41,642.0,28.0,0.25,134.0,0.0,129.0,22.8,29.7,...,4,No,0,Unknown,No,2-High,Suburban,Crafts,No,No
3,3184922,30.18,281.0,30.0,0.00,0.0,0.0,132.0,-0.2,4.0,...,3,No,0,150,No,3-Good,Suburban,Other,Unknown,Yes
4,3350054,179.29,1459.0,95.0,0.50,118.0,48.6,-1360.0,-77.0,21.7,...,4,No,0,Unknown,No,3-Good,Rural,Other,Unknown,Yes


In [4]:
df.duplicated().sum()

0

In [5]:
df.isna().sum()

CustomerID                     0
MonthlyRevenue               120
MonthlyMinutes               120
TotalRecurringCharge         120
DirectorAssistedCalls        120
OverageMinutes               120
RoamingCalls                 120
PercChangeMinutes            290
PercChangeRevenues           290
DroppedCalls                   0
BlockedCalls                   0
UnansweredCalls                0
CustomerCareCalls              0
ThreewayCalls                  0
ReceivedCalls                  0
OutboundCalls                  0
InboundCalls                   0
PeakCallsInOut                 0
OffPeakCallsInOut              0
DroppedBlockedCalls            0
CallForwardingCalls            0
CallWaitingCalls               0
MonthsInService                0
UniqueSubs                     0
ActiveSubs                     0
ServiceArea                   21
Handsets                       1
HandsetModels                  1
CurrentEquipmentDays           1
AgeHH1                       716
AgeHH2    

In [6]:
def get_state(code):
    if not isinstance(code, str) or len(code) < 3:
        return np.nan
    metro_code = code[:3] 
    # Disambiguate 'SAN' based on area code
    if metro_code == 'SAN':
        area_code = code[-3:] if len(code) >= 9 else ''
        if area_code in ['512', '210', '830', '254', '361', '956']:  # Texas area codes
            return 'Texas'
        return 'California'  # Default for 'SAN' in other contexts (e.g., Santa Ana)
    return state_map.get(metro_code, np.nan)

def selectBestFeatures(df, target):
    selected_features = []
    for column in df.columns:
        if column != target:
            corr,pval = stats.pearsonr(df[column],df[target])
            if pval < 0.05:
                selected_features.append(column)
            #print("'{column}' has correlation of '{corr}' and pvalue of '{pval}'".format(column=column,corr=corr,pval=pval))
    return selected_features

def remove_outliers_using_IQR(df,column):
    first_quartile, third_quartile = np.percentile(df[column],25), np.percentile(df[column],75)
    IQR = third_quartile - first_quartile
    floor = first_quartile - 1.5 * IQR
    cieling = third_quartile + 1.5 * IQR
    outlier_mask = (df[column] < floor) | (df[column] > cieling)
    print()
    df = df[~outlier_mask]
    return df

In [7]:
df.drop(columns=['CustomerID'],axis=1,inplace=True)
df['HandsetPrice'] = df['HandsetPrice'].replace('Unknown', np.nan)
df['MaritalStatus'] = df['MaritalStatus'].replace('Unknown', np.nan)
#df = df.dropna()
#df.reset_index(drop=True,inplace=True)
#df['HandsetPrice'] = df['HandsetPrice'].astype(int)
state_map = {
        'NYC': 'New York', 'LAX': 'California', 'CHI': 'Illinois', 'HAR': 'Connecticut',
        'DAL': 'Texas', 'SAN': 'Texas', 'MIA': 'Florida', 'MIL': 'Wisconsin', 'NCR': 'North Carolina',
        'PHI': 'Pennsylvania', 'OKC': 'Oklahoma', 'MIN': 'Minnesota', 'OHI': 'Ohio', 'IND': 'Indiana',
        'SFR': 'California', 'LOU': 'Kentucky', 'APC': 'Maryland', 'BOS': 'Massachusetts', 'NOL': 'Louisiana',
        'SEA': 'Washington', 'DET': 'Michigan', 'PHX': 'Arizona', 'NEV': 'Nevada', 'OMA': 'Nebraska',
        'NNY': 'New York', 'STL': 'Missouri', 'FLN': 'Florida', 'HOU': 'Texas', 'PIT': 'Pennsylvania',
        'KCY': 'Kansas', 'ATL': 'Georgia', 'AIR': 'South Carolina', 'NMX': 'New Mexico', 'SHE': 'Virginia',
        'LAU': 'Mississippi', 'SLC': 'Utah', 'HWI': 'Hawaii', 'NSH': 'Tennessee', 'BIR': 'Alabama',
        'AWI': 'Wisconsin', 'DEN': 'Colorado', 'NOR': 'Minnesota', 'GCW': 'Mississippi', 'NVU': 'Nevada',
        'LAW': 'Texas', 'VAH': 'Virginia', 'SDA': 'South Dakota', 'SEW': 'Oregon', 'HOP': 'Montana',
        'SFU': 'California'
    }

yes_no_features = ['Churn','ChildrenInHH','HandsetRefurbished','HandsetWebCapable','TruckOwner','RVOwner','BuysViaMailOrder',
     'RespondsToMailOffers','OptOutMailings','NonUSTravel','OwnsComputer','HasCreditCard','NewCellphoneUser','NotNewCellphoneUser',
     'OwnsMotorcycle','MadeCallToRetentionTeam']

keep_values = {'CreditRating': ['2-High','1-Highest','3-Good'], 'Occupation': ['Professional','Crafts'], 
               'PrizmCode': ['Suburban','Town'],'ServiceArea': ['Texas', 'New York', 'California', 'Florida',
               'Maryland', 'Illinois','Georgia', 'Ohio', 'North Carolina', 'Nevada', 'Pennsylvania','Michigan', 
               'Massachusetts' ,'Washington', 'Kansas'] }

for feature in yes_no_features:
    df[feature] = [1 if value == 'Yes' else 0 for value in df[feature]]
df['Homeownership'] = [1 if value == 'Known' else 0 for value in df['Homeownership']]
df['ServiceArea'] = df['ServiceArea'].apply(lambda code: get_state(code))
for feature, values in keep_values.items():
    df[feature] = [value if value in values else 'Other' for value in df[feature]]
df = pd.get_dummies(df,columns=['CreditRating','PrizmCode','Occupation','ServiceArea'])

In [8]:
df['MaritalStatus'] = [1 if status == 'Yes' else 0 if status =='No' else np.nan for status in df['MaritalStatus']]

In [14]:
df

,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,BlockedCalls,...,ServiceArea_Massachusetts,ServiceArea_Michigan,ServiceArea_Nevada,ServiceArea_New York,ServiceArea_North Carolina,ServiceArea_Ohio,ServiceArea_Other,ServiceArea_Pennsylvania,ServiceArea_Texas,ServiceArea_Washington
0,39.79,234.0,55.0,0.25,0.0,1.7,31.0,8.8,6.7,0.7,...,0,0,0,0,0,0,0,0,0,0
1,60.19,551.0,60.0,0.74,0.0,0.3,-88.0,-10.2,2.7,32.7,...,0,0,0,0,0,0,0,0,0,0
2,85.41,642.0,28.0,0.25,134.0,0.0,129.0,22.8,29.7,0.0,...,0,0,0,1,0,0,0,0,0,0
3,30.18,281.0,30.0,0.00,0.0,0.0,132.0,-0.2,4.0,0.0,...,0,0,0,1,0,0,0,0,0,0
4,179.29,1459.0,95.0,0.50,118.0,48.6,-1360.0,-77.0,21.7,11.0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40832,19.36,18.0,17.0,0.00,0.0,4.5,-18.0,-9.4,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
40833,50.30,140.0,50.0,0.00,0.0,0.0,-77.0,-0.3,1.3,3.0,...,0,0,0,0,0,0,0,0,0,0
40834,15.73,713.0,10.0,1.73,0.0,0.0,-143.0,0.2,4.0,55.0,...,0,0,0,0,0,0,0,0,0,0
40835,80.69,504.0,50.0,0.00,77.0,0.0,48.0,-30.7,1.7,0.7,...,0,0,0,0,0,0,0,0,1,0


In [15]:
imputer = KNNImputer(weights = 'distance')
imputer.fit(df)
#imputer = joblib.load('NullImputer.h5')
filled_df = imputer.transform(df)

In [16]:
import joblib
joblib.dump(imputer,'NullImputer.h5')

['NullImputer.h5']

In [17]:
imputer = joblib.load('NullImputer.h5')
test = imputer.transform(df)

In [18]:
f = pd.DataFrame(filled_df,columns= df.columns)

In [ ]:
f

In [ ]:
df.shape[0] - 15752

In [ ]:
df['MaritalStatus'].value_counts()

In [ ]:
m = df.drop(columns = ['HandsetPrice'],axis=0)

In [ ]:
m.isna().sum()

In [ ]:
len(selected_features)

In [ ]:
imputer.fit_transform(df)

In [ ]:
imputer = KNNImputer(weights = 'distance')
filled_df = imputer.fit_transform(df)

In [20]:
selected_features = selectBestFeatures(f, 'Churn')

In [21]:
features = f[selected_features]
labels = f['Churn']

x_train, x_validation, y_train, y_validation = train_test_split(features, labels, random_state = 42, stratify = labels, test_size = 0.1)

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=27)
x_train, y_train = sm.fit_resample(x_train, y_train)

C:\Users\Nour Kamaly\anaconda3\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [24]:
classifier = DecisionTreeClassifier(max_depth=8,
        min_samples_split=5,
        min_samples_leaf=2,
        criterion='gini',
        random_state=42)

In [25]:
classifier.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=8, min_samples_leaf=2, min_samples_split=5,
                       random_state=42)

In [31]:
x_train

,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,UnansweredCalls,...,CreditRating_Other,PrizmCode_Suburban,PrizmCode_Town,ServiceArea_Florida,ServiceArea_Kansas,ServiceArea_Maryland,ServiceArea_Michigan,ServiceArea_Ohio,ServiceArea_Texas,ServiceArea_Washington
0,34.460000,198.000000,30.000000,0.000000,0.000000,0.000000,-74.000000,2.000000,2.000000,11.300000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0
1,53.970000,261.000000,60.000000,0.000000,0.000000,3.100000,-115.000000,1.300000,26.700000,29.700000,...,1.000000,1.000000,0.000000,1.0,0.0,0.0,0.00000,0.000000,0.000000,0.0
2,64.360000,453.000000,34.000000,0.000000,86.000000,0.800000,-391.000000,-64.400000,19.000000,28.700000,...,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0
3,35.880000,182.000000,45.000000,0.000000,0.000000,0.200000,76.000000,2.000000,1.300000,5.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,1.000000,0.000000,0.0
4,51.970000,463.000000,39.000000,2.480000,59.000000,0.000000,-120.000000,-24.300000,11.300000,40.000000,...,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52317,28.249781,405.375873,27.919578,0.102981,15.231641,1.539512,9.215030,-20.534265,1.788899,15.659789,...,0.000000,0.000000,0.104021,0.0,0.0,0.0,0.00000,0.104021,0.000000,0.0
52318,46.727837,377.918100,55.701720,0.445905,11.459050,0.000000,-5.400408,-4.631245,9.776036,81.548534,...,0.000000,0.891810,0.000000,0.0,0.0,0.0,0.89181,0.000000,0.000000,0.0
52319,10.514982,88.023760,10.000000,0.498303,0.061098,0.002037,143.928719,-0.423627,0.000000,0.745145,...,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.996606,0.0
52320,65.068954,1.987418,27.352285,0.000000,0.000000,0.592451,-1.987418,-25.571715,0.000000,0.012582,...,0.987418,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0


In [26]:
y_pred = classifier.predict(x_validation)

In [27]:
precision_score(y_validation,y_pred)

0.4384858044164038

In [28]:
recall_score(y_validation,y_pred)

0.11809685641461343

In [29]:
y = classifier.predict(x_train)
precision_score(y_train,y)

0.9110912114621977

In [30]:
recall_score(y_train,y)

0.5444745995948167

In [ ]:
params = classifier.get_params()

In [ ]:
params

In [ ]:
#Models
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier, RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn import tree
from sklearn.linear_model import SGDClassifier,RidgeClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB,GaussianNB,GaussianNB
from sklearn.tree import DecisionTreeClassifier


models = {
    "AdaBoost" : AdaBoostClassifier(DecisionTreeClassifier(),algorithm="SAMME.R",n_estimators=100),
    "GradientBoosting"      : GradientBoostingClassifier(),
    "BaggingClassifier"  : BaggingClassifier(tree.DecisionTreeClassifier(random_state=1)),
    "RandomForest"      : RandomForestClassifier(n_estimators=3),
    "eXtremeGradientBoosting"     : XGBClassifier(objective= 'binary:logistic'),
    #"KNN"                : KNeighborsClassifier(n_neighbors = 30, weights = 'distance'),
    "DecisionTree"      : DecisionTreeClassifier(),
    "ExtraTreeClassifier"       : ExtraTreesClassifier(n_estimators=3)
}


for name, model in models.items():
    

    print(f'Using model: {name}')
    model.fit(x_train, y_train)

    print(f'Training Score: {model.score(x_train, y_train)}')
    print(f'Test Score: {model.score(x_validation, y_validation)}') 
    
    y_pred = model.predict(x_validation)
    print(f'confusion_matrix:\n {confusion_matrix(y_validation, y_pred)}')
    print(f'accuracy_score: {accuracy_score(y_validation, y_pred)}')
    print(f'precision_score: {precision_score(y_validation, y_pred,average="macro")}')
    print(f'recall_score: {recall_score(y_validation, y_pred,average="macro")}')
    print(f'f1_score: {recall_score(y_validation, y_pred,average="macro")}')
    print('-------------')

In [23]:
import optuna
# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    max_depth = trial.suggest_int("max_depth", 1, 8)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 8)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 8)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    
    # Create and train the model
    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        random_state=42
    )
    
    # Use cross-validation to evaluate the model
    scores = cross_val_score(model, x_train, y_train, cv=5, scoring="accuracy")
    return scores.mean()

# Create a study and optimize
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=32)

[I 2025-06-04 20:23:24,893] A new study created in memory with name: no-name-b9505d83-9927-4fe1-b21b-71df84ad823a
[I 2025-06-04 20:23:32,911] Trial 0 finished with value: 0.6568989247688888 and parameters: {'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 5, 'criterion': 'entropy'}. Best is trial 0 with value: 0.6568989247688888.
[I 2025-06-04 20:23:39,771] Trial 1 finished with value: 0.6568989247688888 and parameters: {'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 2, 'criterion': 'entropy'}. Best is trial 0 with value: 0.6568989247688888.
[I 2025-06-04 20:23:46,811] Trial 2 finished with value: 0.6568989247688888 and parameters: {'max_depth': 4, 'min_samples_split': 4, 'min_samples_leaf': 8, 'criterion': 'entropy'}. Best is trial 0 with value: 0.6568989247688888.
[I 2025-06-04 20:23:50,736] Trial 3 finished with value: 0.5931957186544342 and parameters: {'max_depth': 2, 'min_samples_split': 7, 'min_samples_leaf': 5, 'criterion': 'entropy'}. Best is trial 0 wi

In [34]:
from lazypredict.Supervised import LazyClassifier

In [ ]:
from sklearn.metrics import make_scorer, f1_score

def cutom_f1(true,predictions):
    f1 = f1_score(true, predictions)
    return 'macro_f1', f1, True
# Define your custom metric
f1 = make_scorer(f1_score, greater_is_better = True)

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=f1)

# Fit and evaluate models
models, predictions = clf.fit(x_train, x_validation, y_train, y_validation)

models

  0%|          | 0/32 [00:00<?, ?it/s]

In [36]:
models

NameError: name 'models' is not defined